### Install needed dependencies

In [ ]:
conda install pytorch=1.1.0 torchvision cudatoolkit=10.0 -c pytorch
pip install tensorflow-gpu==1.14.0
pip install -r requirements.txt

### Data preparation for supervised training

In [11]:
# place Adobe Matting Dataset zip here and run the cell

import os
import zipfile


if os.path.exists("Adobe_Deep_Matting_Dataset.zip"):
    with zipfile.ZipFile("Adobe_Deep_Matting_Dataset.zip") as f:
        f.extractall(".")

if os.path.exists("Adobe_Deep_Matting_Dataset.zip"):
    os.remove("Adobe_Deep_Matting_Dataset.zip")

In [15]:
# create combined Dataset

%cd Data_Adobe
!./prepare.sh ../Combined_Dataset

In [16]:
# Getting MS-COCO images for backgrounds

!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/test2017.zip

!unzip train2017.zip
!unzip test2017.zip

!mv train2017 bg_train
!mv test2017 bg_test

Streaming output truncated to the last 5000 lines.
 extracting: test2017/000000145034.jpg  
 extracting: test2017/000000234833.jpg  
 extracting: test2017/000000025927.jpg  
 extracting: test2017/000000393116.jpg  
 extracting: test2017/000000536478.jpg  
 extracting: test2017/000000365460.jpg  
 extracting: test2017/000000089887.jpg  
 extracting: test2017/000000274928.jpg  
 extracting: test2017/000000310028.jpg  
 extracting: test2017/000000229793.jpg  
 extracting: test2017/000000394339.jpg  
 extracting: test2017/000000564538.jpg  
 extracting: test2017/000000440585.jpg  
 extracting: test2017/000000541512.jpg  
 extracting: test2017/000000395525.jpg  
 extracting: test2017/000000235435.jpg  
 extracting: test2017/000000069964.jpg  
 extracting: test2017/000000131962.jpg  
 extracting: test2017/000000281392.jpg  
 extracting: test2017/000000534893.jpg  
 extracting: test2017/000000128676.jpg  
 extracting: test2017/000000187639.jpg  
 extracting: test2017/000000153095.jpg  
 extra

In [36]:
# composing foreground and background

!python compose.py --workers 8 --fg_path fg_train --mask_path mask_train --bg_path bg_train --out_path merged_train --out_csv Adobe_train_data.csv
!python compose.py --workers 8 --fg_path fg_test --mask_path mask_test --bg_path bg_test --out_path merged_test

Streaming output truncated to the last 5000 lines.
(150) ...al-park-1172381_1920.jpg:  22% 22/100 [00:40<02:29,  1.92s/it]
(150) ...al-park-1172381_1920.jpg:  22% 22/100 [00:40<02:29,  1.92s/it]
(150) ...al-park-1172381_1920.jpg:  23% 23/100 [00:42<02:26,  1.90s/it]
(150) ...al-park-1172381_1920.jpg:  23% 23/100 [00:42<02:26,  1.90s/it]
(150) ...al-park-1172381_1920.jpg:  24% 24/100 [00:43<02:15,  1.78s/it]
(150) ...al-park-1172381_1920.jpg:  24% 24/100 [00:43<02:15,  1.78s/it]
(150) ...al-park-1172381_1920.jpg:  25% 25/100 [00:46<02:21,  1.88s/it]
(150) ...al-park-1172381_1920.jpg:  25% 25/100 [00:46<02:21,  1.88s/it]
(150) ...al-park-1172381_1920.jpg:  26% 26/100 [00:47<02:13,  1.80s/it]
(150) ...al-park-1172381_1920.jpg:  26% 26/100 [00:47<02:13,  1.80s/it]
(150) ...al-park-1172381_1920.jpg:  27% 27/100 [00:49<02:16,  1.88s/it]
(150) ...al-park-1172381_1920.jpg:  27% 27/100 [00:49<02:16,  1.88s/it]
(150) ...al-park-1172381_1920.jpg:  28% 28/100 [00:51<02:19,  1.94s/it]
(150) ...al-p

### Supervised training

In [ ]:
# training

%cd ..
!python train_adobe.py -n Adobe_train -bs 4 -res 512

### Data preparation for unsupervised training

In [14]:
# splitting videos on frames
# get data here https://drive.google.com/drive/folders/1j3BMrRFhFpfzJAe6P2WDtfanoeSCLPiq?usp=sharing

!python frames_split.py -f Captured_Data/fixed-camera/train -e mp4

/content/gdrive/.shortcut-targets-by-id/1j3BMrRFhFpfzJAe6P2WDtfanoeSCLPiq/Captured_Data/fixed-camera/train/rml.mp4
VIDIOC_REQBUFS: Function not implemented
False
Traceback (most recent call last):
  File "frames_split.py", line 40, in <module>
    frames_from_video(folder / file)
  File "frames_split.py", line 21, in frames_from_video
    cv2.imwrite(str(output / ("%04d_img.png" % count)), image)
cv2.error: OpenCV(4.1.2) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'



In [ ]:
# getting tensorflow version of Deeplabv3+

!git clone https://github.com/tensorflow/models.git
%cd models/research/
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
%cd ../..

In [15]:
# exctacting masks

!./apply_segmentation.sh Captured_Data/fixed-camera/train

In [ ]:
# prepare real data

!python prepare_real.py

### Unsupervised training

In [ ]:
# training

!python train_real_fixed.py -n Real_fixed -bs 4 -res 512 -init_model Models/syn-comp-adobe-trainset/net_epoch_64.pth